# Collecting and Organizing Data

In [1]:
import csv
import os
import pandas as pd
from pathlib import Path

First we have to make a master dataframe with all the relevant data. This master dataframe will contain an entry for every single onset, for every single wav file in the audio file. If an audio file is multiple drum sounds, then there is a single onset for each drum sound, and an single audio file will contirnbute to multiple entries in the dataset. We will have to parse AVP and LVT seperately and then combine them. 

In [2]:
# function to parse AVP csv, get the onset time, instrument label, onset phoneme, coda phoneme, dataset, participant id, subset, csv file path, wav file path
def parse_avp_csv(csv_path):
    """
    Parses an AVP CSV with no header, returning a list of dicts.
    Each dict has:
      - onset_time (float)
      - instrument_label (str)
      - onset_phoneme (str)
      - coda_phoneme (str)
      - dataset (str) = "AVP"
      - participant_id (str)
      - subset (str) = "personal"
      - csv_file_path (str)
      - wav_file_path (str)
    """
    # 1) Extract some metadata from the file path
    csv_dir = os.path.dirname(csv_path)             # e.g., "AVP_Dataset/Personal/Participant_1"
    csv_file_name = os.path.basename(csv_path)      # e.g., "P1_kd.csv" or "Participant_1_kd.csv"
    base_name, _ = os.path.splitext(csv_file_name)  # e.g., "P1_kd" or "Participant_1_kd"
    
    # 2) Determine participant_id from the file name
    #    Suppose your file name is "P1_kd.csv", so the participant part is "P1".
    #    If it’s "Participant_1_kd.csv", you might want the first two segments. Adapt as needed.
    #    Here's a simple approach:
    parts = base_name.split("_")  # e.g. ["P1", "kd"] or ["Participant", "1", "kd"]
    participant_id = parts[0]     # e.g. "P1" or "Participant"
    # If your actual naming is more complicated, tweak the logic. 
    # For instance, if "Participant_1" is always 2 segments, you might do participant_id = "_".join(parts[:2])
    
    # 3) Build the wav path. If the CSV is "P1_kd.csv", the wav is "P1_kd.wav" in the same folder.
    wav_file_name = base_name + ".wav"
    wav_file_path = os.path.join(csv_dir, wav_file_name)
    
    # 4) We'll fix "dataset" = "AVP" and "subset" = "personal" 
    #    (since that's the arrangement you described for the AVP dataset).
    dataset = "AVP"
    subset = "personal"
    
    # 5) Parse each row of the CSV
    data = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            # Expecting: [onset_time, instrument_label, onset_phoneme, coda_phoneme]
            if len(row) < 2:
                continue  # skip empty or malformed lines
            onset_time = float(row[0])
            instrument_label = row[1]
            onset_phoneme = row[2] if len(row) > 2 else ''
            coda_phoneme = row[3] if len(row) > 3 else ''
            
            entry = {
                'onset_time': onset_time,
                'instrument_label': instrument_label,
                'onset_phoneme': onset_phoneme,
                'coda_phoneme': coda_phoneme,
                'dataset': dataset,
                'participant_id': participant_id,
                'subset': subset,
                'csv_file_path': csv_path,
                'wav_file_path': wav_file_path
            }
            data.append(entry)
    
    return data

In [3]:
#function to collect all AVP data, takes a root directory as input, walks through the AVP dataset directory and collects all CSV data into a master DataFrame
def collect_all_avp_data(root_dir):
    """
    Walks through the AVP dataset directory and collects all CSV data into a master DataFrame,
    maintaining the grouping of entries from the same CSV file
    """
    all_data = []
    personal_dir = os.path.join(root_dir, "Personal")
    
    # Walk through all participant directories in sorted order
    for participant_dir in sorted(os.listdir(personal_dir)):
        participant_path = os.path.join(personal_dir, participant_dir)
        
        # Skip if not a directory or hidden files
        if not os.path.isdir(participant_path) or participant_dir.startswith('.'):
            continue
            
        # Process CSV files in sorted order
        for file_name in sorted(os.listdir(participant_path)):
            if file_name.endswith('.csv'):
                csv_path = os.path.join(participant_path, file_name)
                
                try:
                    parsed_data = parse_avp_csv(csv_path)
                    # Add the source filename as a field for sorting
                    for entry in parsed_data:
                        entry['source_file'] = file_name
                    all_data.extend(parsed_data)
                except Exception as e:
                    print(f"Error processing {csv_path}: {str(e)}")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort to maintain grouping:
    # First by participant_id, then by source file, then by onset_time
    df = df.sort_values(['participant_id', 'source_file', 'onset_time'])
    
    # Optionally remove the temporary source_file column if you don't need it
    df = df.drop('source_file', axis=1)
    
    return df

In [4]:
# def get_standardized_phoneme(simple_phoneme, is_onset=True):
#     """
#     Converts LVT phoneme notation to match AVP's IPA notation
    
#     Args:
#         simple_phoneme: The phoneme from LVT dataset
#         is_onset: Boolean indicating if this is an onset (True) or coda (False) phoneme
#     """
#     # Onset phonemes (consonants at start)
#     onset_map = {
#         '!': '!',        # keep as is (appears in both)
#         'k': 'k',        # keep as is
#         'p': 'p',        # keep as is
#         's': 's',        # keep as is
#         't': 't',        # keep as is
#         'ts': 'ts',      # keep as is
#         'tʃ': 'tʃ',      # keep as is
#         'ʔ': 'tʃ',       # map glottal stop to tʃ
#         'ʡʢ': 'ʡʢ'       # keep as is (appears in both)
#     }
    
#     # Coda phonemes (vowels/endings)
#     coda_map = {
#         'a': 'æ',        # map 'a' to 'æ' as it's more common in AVP
#         'h': 'h',        # keep as is
#         'u': 'u',        # keep as is
#         'x': 'x',        # keep as is
#         'ʊ': 'ʊ'         # keep as is
#     }
    
#     # Choose which mapping to use
#     phoneme_map = onset_map if is_onset else coda_map
    
#     # Return mapped phoneme if it exists, otherwise return original
#     return phoneme_map.get(simple_phoneme, simple_phoneme)

In [5]:
#functiuon to parse LVT csv, get the onset time, instrument label, onset phoneme, coda phoneme, dataset, participant id, subset, csv file path, wav file path
def parse_lvt_csv(csv_path):
    """
    Parses an LVT CSV with no header, returning a list of dicts.
    Similar to parse_avp_csv but handles LVT-specific formatting.
    """
    # Extract metadata from the file path
    csv_dir = os.path.dirname(csv_path)             
    csv_file_name = os.path.basename(csv_path)      # e.g., "AFRP.csv"
    base_name, _ = os.path.splitext(csv_file_name)  # e.g., "AFRP"
    
    # Determine if this is from Frase or Improviso folder
    subset = "Frase" if "Frase" in csv_dir else "Improviso"
    
    # Participant ID is the filename without extension
    participant_id = base_name
    
    # Build the wav path (add "3" before .wav)
    wav_file_name = base_name + "3.wav"
    wav_file_path = os.path.join(csv_dir, wav_file_name)
    
    # Mapping for instrument labels
    instrument_map = {
        "Kick": "kd",
        "Snare": "sd",
        "HH": "hhc"  # Assuming all HH in LVT are closed hi-hats
    }
    
    data = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) < 2:
                continue  # skip empty or malformed lines
                
            onset_time = float(row[0])
            original_label = row[1]
            instrument_label = instrument_map.get(original_label, original_label)
            onset_phoneme = row[2] if len(row) > 2 else ''
            coda_phoneme = row[3] if len(row) > 3 else ''
            
            # onset_phoneme = get_standardized_phoneme(row[2], is_onset=True)   # converts 'ts' if needed
            # coda_phoneme = get_standardized_phoneme(row[3], is_onset=False)   # converts 'x' if needed
            
            entry = {
                'onset_time': onset_time,
                'instrument_label': instrument_label,
                'onset_phoneme': onset_phoneme,
                'coda_phoneme': coda_phoneme,
                'dataset': "LVT",
                'participant_id': participant_id,
                'subset': subset,
                'csv_file_path': csv_path,
                'wav_file_path': wav_file_path
            }
            data.append(entry)
    
    return data

In [6]:
#function to collect all LVT data, takes a root directory as input, walks through the LVT dataset directory and collects all CSV data into a master DataFrame
def collect_all_lvt_data(root_dir):
    """
    Walks through the LVT dataset directory and collects all CSV data into a master DataFrame
    """
    all_data = []
    
    # Process both Frase and Improviso folders
    for subset_dir in ["Frase", "Improviso"]:
        subset_path = os.path.join(root_dir, subset_dir)
        
        # Skip if directory doesn't exist
        if not os.path.isdir(subset_path):
            continue
            
        # Process CSV files in sorted order
        for file_name in sorted(os.listdir(subset_path)):
            if file_name.endswith('.csv') and not file_name.startswith('.'):
                csv_path = os.path.join(subset_path, file_name)
                
                try:
                    parsed_data = parse_lvt_csv(csv_path)
                    # Add source file for sorting
                    for entry in parsed_data:
                        entry['source_file'] = file_name
                    all_data.extend(parsed_data)
                except Exception as e:
                    print(f"Error processing {csv_path}: {str(e)}")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort to maintain grouping
    df = df.sort_values(['subset', 'participant_id', 'source_file', 'onset_time'])
    
    # Remove temporary sorting column
    df = df.drop('source_file', axis=1)
    
    return df

In [7]:
#function to create all datasets, calls the functions to collect AVP and LVT data and then combines them into a master dataframe
def create_all_datasets():
    avp_dataset_path = "../AVP-LVT_Dataset/AVP_Dataset"
    lvt_dataset_path = "../AVP-LVT_Dataset/LVT_Dataset"
    
    # Collect data from both datasets
    print("Processing AVP dataset...")
    avp_df = collect_all_avp_data(avp_dataset_path)
    
    print("Processing LVT dataset...")
    lvt_df = collect_all_lvt_data(lvt_dataset_path)
    
    # Save individual datasets
    print("\nSaving individual datasets...")
    avp_df.to_csv('EDA/avp_dataset.csv', index=False)
    lvt_df.to_csv('EDA/lvt_dataset.csv', index=False)
    
    # Combine and save master dataset
    print("Creating and saving master dataset...")
    master_df = pd.concat([avp_df, lvt_df], ignore_index=True)
    master_df.to_csv('EDA/master_dataset.csv', index=False)
    
    # Print summary statistics
    print("\nDataset Summaries:")
    print(f"AVP Dataset: {len(avp_df)} events")
    print("\nAVP participants:", len(avp_df['participant_id'].unique()))
    print("AVP instrument distribution:")
    print(avp_df['instrument_label'].value_counts())
    
    print(f"\nLVT Dataset: {len(lvt_df)} events")
    print("LVT subsets:", lvt_df['subset'].unique())
    print("LVT participants:", len(lvt_df['participant_id'].unique()))
    print("LVT instrument distribution:")
    print(lvt_df['instrument_label'].value_counts())
    
    print(f"\nMaster Dataset: {len(master_df)} total events")
    print("Distribution by dataset:")
    print(master_df['dataset'].value_counts())
    
    return avp_df, lvt_df, master_df


In [8]:
avp_df, lvt_df, master_df = create_all_datasets()

Processing AVP dataset...
Processing LVT dataset...

Saving individual datasets...
Creating and saving master dataset...

Dataset Summaries:
AVP Dataset: 4873 events

AVP participants: 28
AVP instrument distribution:
instrument_label
kd     1447
sd     1253
hhc    1164
hho    1009
Name: count, dtype: int64

LVT Dataset: 841 events
LVT subsets: ['Frase' 'Improviso']
LVT participants: 40
LVT instrument distribution:
instrument_label
hhc    334
kd     329
sd     178
Name: count, dtype: int64

Master Dataset: 5714 total events
Distribution by dataset:
dataset
AVP    4873
LVT     841
Name: count, dtype: int64


Now we have master_dataset.csv, and master_df, both of which contain the info for every single onset for every single sound in the dataset. 

In [9]:
def analyze_phonemes():
    """
    Analyze and compare phonemes between AVP and LVT datasets
    """
    # Read both datasets
    avp_df = pd.read_csv('EDA/avp_dataset.csv')
    lvt_df = pd.read_csv('EDA/lvt_dataset.csv')
    
    print("AVP Unique Onset Phonemes:")
    print(sorted(avp_df['onset_phoneme'].unique()))
    print("\nAVP Unique Coda Phonemes:")
    print(sorted(avp_df['coda_phoneme'].unique()))
    
    print("\nLVT Unique Onset Phonemes:")
    print(sorted(lvt_df['onset_phoneme'].unique()))
    print("\nLVT Unique Coda Phonemes:")
    print(sorted(lvt_df['coda_phoneme'].unique()))

# Run the analysis
analyze_phonemes()

AVP Unique Onset Phonemes:
['!', 'dʒ', 'k', 'kg', 'kʃ', 'p', 's', 't', 'ts', 'tɕ', 'tʃ', 'tʒ', 'ʡʢ']

AVP Unique Coda Phonemes:
['I', 'a', 'e', 'h', 'i', 'o', 'u', 'x', 'æ', 'œ', 'ɐ', 'ɘ', 'ə', 'ɪ', 'ɯ', 'ʊ', 'ʌ']

LVT Unique Onset Phonemes:
['!', 'k', 'p', 's', 't', 'ts', 'tʃ', 'ʔ', 'ʡʢ']

LVT Unique Coda Phonemes:
['a', 'h', 'u', 'x', 'ʊ']


In [18]:
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Try using soundfile directly instead of librosa.load
def segment_audio(master_df, output_dir, segment_duration=0.5):
    """
    Segments audio files and saves them to disk.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    segment_info = []
    
    # Group by wav_file_path
    for wav_path, group in tqdm(master_df.groupby('wav_file_path')):
        print(f"\nProcessing {wav_path}")
        
        try:
            # Use soundfile instead of librosa.load
            y, sr = sf.read(wav_path)
            
            # Process each onset in this file
            for idx, row in group.iterrows():
                start_sample = int(row['onset_time'] * sr)
                end_sample = start_sample + int(segment_duration * sr)
                
                # Handle edge cases
                if start_sample < 0:
                    start_sample = 0
                if end_sample > len(y):
                    end_sample = len(y)
                
                if end_sample > start_sample:
                    segment = y[start_sample:end_sample]
                    
                    # Pad if needed
                    if len(segment) < int(segment_duration * sr):
                        segment = np.pad(segment, 
                                      (0, int(segment_duration * sr) - len(segment)),
                                      mode='constant')
                    
                    segment_filename = (f"{row['dataset']}_{row['participant_id']}_"
                                     f"{row['instrument_label']}_{idx:04d}.wav")
                    
                    segment_path = output_dir / segment_filename
                    sf.write(str(segment_path), segment, sr)
                    
                    segment_info.append({
                        'segment_path': str(segment_path),
                        'instrument_label': row['instrument_label'],
                        'participant_id': row['participant_id'],
                        'dataset': row['dataset'],
                        'original_wav': wav_path,
                        'onset_time': row['onset_time']
                    })
        
        except Exception as e:
            print(f"Error processing {wav_path}: {str(e)}")
            continue
    
    segment_df = pd.DataFrame(segment_info)
    segment_df.to_csv('segment_info.csv', index=False)
    
    return segment_df
    

In [19]:
master_df = pd.read_csv('EDA/master_dataset.csv')
    
# Segment all audio files
print("Starting audio segmentation...")
segment_df = segment_audio(master_df, output_dir='segments')
    
# Print summary
print("\nSegmentation Summary:")
print(f"Total segments extracted: {len(segment_df)}")
print("\nInstrument distribution:")
print(segment_df['instrument_label'].value_counts())
print("\nDataset distribution:")
print(segment_df['dataset'].value_counts())

Starting audio segmentation...


  4%|▍         | 8/180 [00:00<00:02, 78.79it/s]


Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_1/P1_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_1/P1_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_1/P1_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_1/P1_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_1/P1_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_10/P10_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_10/P10_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_10/P10_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_10/P10_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_10/P10_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_11/P11_HHclosed_Pers

 15%|█▌        | 27/180 [00:00<00:01, 86.26it/s]


Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_12/P12_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_12/P12_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_12/P12_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_12/P12_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_13/P13_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_13/P13_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_13/P13_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_13/P13_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_13/P13_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_14/P14_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_14/P14_HHo

...


Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_6/P6_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_6/P6_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_6/P6_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_6/P6_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_7/P7_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_7/P7_HHopened_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_7/P7_Improvisation_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_7/P7_Kick_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_7/P7_Snare_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_8/P8_HHclosed_Personal.wav

Processing ../AVP-LVT_Dataset/AVP_Dataset/Personal/Participant_8/P8_HHopened_Personal.wav

Pr

 86%|████████▌ | 155/180 [00:01<00:00, 96.09it/s]


Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/AFRP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/AZiP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/BeaP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/BicP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/CatP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/CavP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/CraP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/IsaP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/JOlP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/JSiP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/JoSP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/MCoP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/MafP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/MarP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/NorP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/RicP3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Frase/RobP3.w

100%|██████████| 180/180 [00:02<00:00, 81.43it/s] 


Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/CraI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/IsaI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/JOlI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/JSiI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/JSoI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/MCoI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/MafI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/MarI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/NorI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/RicI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/RobI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/SofI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/ZgaI3.wav

Processing ../AVP-LVT_Dataset/LVT_Dataset/Improviso/ZizI3.wav

Segmentation Summary:
Total segments extracted: 5714

Instrument distribution:
instrument_label
kd     1776
hhc    14